<a href="https://colab.research.google.com/github/Rvp127/Healthcare_chat-bot-using-neural-network/blob/main/chat_bot_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING LIBRARIES

In [28]:
import random
import json
import pickle
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


LOAD THE TRAINING DATA

In [29]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open("intents.json").read())

words = [] 
classes = [] 
documents = [] 
ignore_letters = ["?", "!", ".", ","] 

for intent in intents["intents"]:
    for pattern in intent["patterns"]:  
        word_list = nltk.word_tokenize(pattern) 
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        if intent["tag"] not in classes: 
           classes.append(intent["tag"]), 
print(documents) 


[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going'], 'greetings'), (['sup'], 'greetings'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['ciao'], 'goodbye'), (['thank', 'you'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'i', 'call', 'you'], 'name'), (['what', "'s", 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['can', 'you', 'tell', 'me', 'your', 'name'], 'name'), (['i', 'have', 'Runny', 'or', 'stuffy', 'nose'], 'common cold symptoms'), (['i', 'have', 'Sore', 'throat'], 'common cold symptoms'), (['i', 'have', 'Cough'], 'common cold symptoms'), (['i', 'h

PREPARING THE TRAINING DATA

In [30]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters] 
words = sorted(set(words)) 
print(words) 


["'m", "'s", '(', ')', 'Anxiety', 'Chills', 'Congestion', 'Cough', 'Dehydration', 'Generally', 'Headache', 'Hopeless', 'How', 'I', 'Increased', 'Irritability', 'Looking', 'Loss', 'Lost', 'Low-grade', 'Muscle', 'Runny', 'Slight', 'Sneezing', 'Sore', 'Sweating', 'What', 'a', 'ache', 'age', 'am', 'an', 'and', 'any', 'appetite', 'appointment', 'are', 'asthma', 'at', 'available', 'better', 'blurry', 'body', 'breath', 'buy', 'bye', 'call', 'can', 'change', 'chest', 'ciao', 'cold', 'common', 'consult', 'consultation', 'consulting', 'contact', 'coughing', 'cya', 'day', 'death', 'depressed', 'depression', 'diabetes', 'difficulty', 'do', 'doctor', 'drink', 'eat', 'experiencing', 'extreme', 'fatigue', 'feel', 'feeling', 'fever', 'flu', 'for', 'frequent', 'from', 'getting', 'going', 'good', 'goodbye', 'got', 'greeting', 'hate', 'have', 'having', 'headache', 'hello', 'help', 'hey', 'hi', 'how', 'hunger', 'i', 'if', 'in', 'increased', 'interest', 'is', 'it', 'keep', 'later', 'like', 'loss', 'lot', '

In [31]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

print(classes)

['Asthma symptoms', 'Consultation for asthma', 'Consultation for common cold', 'Consultation for depression', 'Consultation for diabetes', 'Consultation for fever', 'Depression symptoms', 'Diabetes symptoms', 'age', 'asthma prevention', 'common cold prevention', 'common cold symptoms', 'depression prevention', 'diabetes prevention', 'fever prevention', 'fever symptoms', 'goodbye', 'greetings', 'name']


In [32]:
#saving the classes and words in seperate files using pickle
pickle.dump(words, open('words.pkl', 'wb')) 
pickle.dump(classes, open('classes.pkl', 'wb'))


In [33]:
training = [] 
output_empty = [0] * len(classes) 

for document in documents:
    bag = [] 
    word_patterns = document[0] 
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns] 
    for word in words: 
        bag.append(1) if word in word_patterns else bag.append(0) 
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1 
    training.append([bag, output_row]) 

random.shuffle(training)   
training = np.array(training)  

train_x = list(training[:, 0])  
train_y = list(training[:, 1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


BUILDING THE NEURAL NETWORK

In [34]:
model = Sequential() 
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu')) 
model.add(Dropout(0.5))  
model.add(Dense(64, activation='relu')) 
model.add(Dropout(0.5))  
model.add(Dense(len(train_y[0]), activation='softmax'))    

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) 
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy']) 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1) 
model.save('chatbotmodel.h5', hist) 
print("Done") 

Epoch 1/200


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


26/26 [==============================] - 1s 2ms/step - loss: 2.9982 - accuracy: 0.0556
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 2.8742 - accuracy: 0.1032
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 2.7894 - accuracy: 0.1190
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 2.6744 - accuracy: 0.1349
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 2.4629 - accuracy: 0.1984
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 2.2343 - accuracy: 0.3175
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 2.2432 - accuracy: 0.2540
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 2.0414 - accuracy: 0.3333
Epoch 9/200
26/26 [==============================] - 0s 1ms/step - loss: 1.8568 - accuracy: 0.3730
Epoch 10/200
26/26 [==============================] - 0s 2ms/step - loss: 1.7697 - accuracy: 0.4286
Epoch 11/200
26/26 [=

BUILDING THE CHATBOT

In [35]:
import random
import json
import pickle
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

!pip install tensorflow
from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
lemmatizer = WordNetLemmatizer() 
intents = json.loads(open("intents.json").read())

words = pickle.load(open('words.pkl', 'rb')) 
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbotmodel.h5') 

#function for cleaning up the sentences
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence) 
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words] 
    return sentence_words #This is how we clean up a sentence 

#function for BOW. this will convert a sentence into a list full of zeros and ones that indicate if the word is there or not.
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words) 
    for w in sentence_words:
        for i,word in enumerate(words):
            if word == w:
                bag[i] = 1 
    return np.array(bag) 

#function for predicting where we use the previous 2 functions
def predict_class(sentence): 
    bow = bag_of_words(sentence) 
    res = model.predict(np.array([bow]))[0] 
    ERROR_THRESHOLD = 0.25 
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD] 

#sort the results
    results.sort(key=lambda x: x[1], reverse=True) 
    return_list = [] 
    for r in results: 
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])}) 
    return return_list 

#function to get responses form the bot!
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("GO! Bot is running!")

while True:
  
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)    

GO! Bot is running!
hello
hey!
im coughing a lot and having chest pain
It seem that you are suffering from Asthma
what to do for asthma
medicines you can consume : epinephrine,anticholinergic,Proair HFA, prevention that you must follow : Identify Asthma Triggers, Stay Away From Allergens,Avoid Smoke of Any Type,SUGGESTED FOODS ARE:carrots,juice,eggs,broccoli,cantaloupe,milk
can you recommend a doctor for my asthma
You can contact various doctors here for any kind of consultation: 1. https://www.practo.com/bangalore/doctors-for-bronchial-asthma-treatment, or you can pay a visit to your local area doctor or family doctor.
thank you see ya later
it's been a pleasure
